In [1]:
# data cleaning
import json

def clean_verdicts(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 删除指定字段
    for entry in data:
        entry.pop("human_verdict", None)
        entry.pop("ai_verdicts", None)

    # 保存到新文件
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# 使用示例
input_file = '/content/drive/MyDrive/detection/pre2020_human_text_by_genre/merged_corpus_withOpenai.json'       # 替换为你的输入文件路径
output_file = '/content/drive/MyDrive/detection/pre2020_human_text_by_genre/output_cleaned.json'  # 输出文件路径
clean_verdicts(input_file, output_file)


In [7]:
# run through stealthgpt

import json
import requests
import os
from tqdm import tqdm

# ======== 配置路径和 token ========
input_file = '/content/drive/MyDrive/detection/pre2020_human_text_by_genre/output_cleaned.json'
output_file = '/content/drive/MyDrive/detection/pre2020_human_text_by_genre/output_withStealthgpt.json'
api_token = 'd5d63c6e2ef2f1f666cb61b72819c228f398aa7baf4d724a54700e93c938ee29'  # 🔐 替换为你的实际 token

API_URL = 'https://stealthgpt.ai/api/stealthify'
MAX_PROMPT_LENGTH = 3000

# ======== 加载输入数据 ========
with open(input_file, 'r') as f:
    input_data = json.load(f)

# ======== 加载已处理数据（如果存在） ========
if os.path.exists(output_file):
    with open(output_file, 'r') as f:
        output_data = json.load(f)
else:
    output_data = []

# ======== 构建已处理 topic 索引集合，避免重复 ========
processed_topics = set(entry.get('topic') for entry in output_data)

# ======== 逐条处理并实时保存 ========
for entry in tqdm(input_data, desc="Rephrasing entries"):
    topic = entry.get('topic', '')
    if topic in processed_topics:
        continue  # 跳过已处理

    original_text = entry.get("ai_generated", {}).get("openai", "")
    if not original_text:
        continue

    if len(original_text) > MAX_PROMPT_LENGTH:
        original_text = original_text[:MAX_PROMPT_LENGTH]

    payload = {
        "prompt": original_text,
        "rephrase": True,
        "tone": "Standard",
        "mode": "Medium",
        "business": False,
        "isMultilingual": True
    }

    headers = {
        "api-token": api_token,
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json().get("result", "")
        entry["stealthgpt_rephrased"] = result
        print(f"✅ Success: {topic[:50]}...")

    except requests.RequestException as e:
        print(f"❌ Failed: {topic[:50]}...: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print("🔍 Response text:", e.response.text)
        entry["stealthgpt_rephrased"] = None

    # ==== 每处理一条就追加写入输出文件 ====
    output_data.append(entry)
    with open(output_file, 'w') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)

print(f"✅ All done! Saved to: {output_file}")


Rephrasing entries:  29%|██▉       | 609/2100 [00:04<00:10, 136.95it/s]

✅ Success: Review of Baby Warming Dish: Small size, not enoug...
✅ Success: Poor print quality of a book....
✅ Success: Generator for circuits project with loose wire con...


Rephrasing entries:  29%|██▉       | 612/2100 [00:19<01:04, 23.25it/s] 

✅ Success: Critique of Adam Sandler's acting and writing abil...


Rephrasing entries:  29%|██▉       | 613/2100 [00:25<01:31, 16.22it/s]

...

❌ Failed: Duchess of Deanshire's Party Invitation and Inner ...: 402 Client Error: Payment Required for url: https://www.stealthgpt.ai/api/stealthify
🔍 Response text: {"message":"Not enough credits"}


Rephrasing entries:  64%|██████▍   | 1346/2100 [1:10:05<07:54,  1.59it/s]

❌ Failed: Struggling with identity and future plans at Briar...: 402 Client Error: Payment Required for url: https://www.stealthgpt.ai/api/stealthify
🔍 Response text: {"message":"Not enough credits"}


Rephrasing entries:  64%|██████▍   | 1346/2100 [1:10:05<39:15,  3.12s/it]


KeyboardInterrupt: 

In [20]:
# run through pengram
import json
import requests
from tqdm import tqdm

# === Load Pangram API key ===
api_keys = {
    "pengram": "78019670-daaf-4b40-9700-32ebe2569644"
}

# === Pangram detection function ===
def detect_ai_pangram(text):
    if not text.strip():
        return None
    headers = {
        "Content-Type": "application/json",
        "x-api-key": api_keys["pengram"]
    }
    payload = {"text": text}
    try:
        response = requests.post("https://text.api.pangramlabs.com", headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        return {
            "ai_likelihood": result.get("ai_likelihood"),
            "prediction": result.get("prediction")
        }
    except Exception as e:
        return {"error": str(e)}

# === Load input data ===
input_path = "/content/drive/MyDrive/detection/pre2020_human_text_by_genre/output_withStealthgpt.json"
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# === Run detection ===
filtered_data = []  # 新列表，只保留处理过的记录

for item in tqdm(data, desc="Running Pangram detection"):
    ai_text = item.get("stealthgpt_rephrased", "")
    if not ai_text or ai_text is None:
        continue  # 跳过没有 rephrased 文本的项

    # 原始文本
    human_text = item.get("text", "")
    item["pengram_human_verdict"] = detect_ai_pangram(human_text)
    item["pengram_stealthgpt_verdict"] = detect_ai_pangram(ai_text)

    filtered_data.append(item)  # 加入结果集

# === Save output ===
output_path = input_path.replace(".json", "_pangram_only.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=2)

print("✅ Pangram-only detection completed. Results saved to:", output_path)


Running Pangram detection: 100%|██████████| 1539/1539 [36:42<00:00,  1.43s/it]

✅ Pangram-only detection completed. Results saved to: /content/drive/MyDrive/detection/pre2020_human_text_by_genre/output_withStealthgpt_pangram_only.json


In [4]:

# Reverse-engineer the threshold based on the false positive (FP) rate (pengram)
import json
import numpy as np

def compute_pengram_threshold(data, fpr_target=0.01): #change the ideal fp rate here
    # 直接提取 pengram ai_likelihood 分数
    human_scores = []
    for item in data:
        try:
            score = item['pengram_human_verdict']['ai_likelihood']
            human_scores.append(score)
        except KeyError:
            continue  # 跳过结构缺失的数据项

    if not human_scores:
        raise ValueError("❌ error")

    # 排序并确定 1% FPR 对应的阈值
    human_scores_sorted = np.sort(human_scores)
    cutoff_index = int(np.floor((1 - fpr_target) * len(human_scores_sorted))) - 1
    threshold = human_scores_sorted[cutoff_index]

    print(f"✅ pengram threshold is：{threshold:.8f}")
    return threshold

# 加载数据
with open('/content/drive/MyDrive/detection/pre2020_human_text_by_genre/output_withStealthgpt_pangram_only_cleaned.json', 'r', encoding='utf-8') as f:
    dataset = json.load(f)

compute_pengram_threshold(dataset)


✅ pengram threshold is：0.00165939


np.float64(0.001659393310546875)

In [5]:
# verdict text(human vs ai) by default threshold
import json

# 阈值设定
THRESHOLD = 0.00165939

# 文件路径
input_file = "/content/drive/MyDrive/detection/pre2020_human_text_by_genre/output_withStealthgpt_pangram_only_cleaned.json"        # ← 替换为你的 JSON 文件名
output_file = "/content/drive/MyDrive/detection/pre2020_human_text_by_genre/output_withStealthgpt_pangram_default.json"

# 读取 JSON 数据
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# 添加两个标签字段
for entry in data:
    # 初始化为 None，避免 KeyError 崩溃
    entry["label_human"] = None
    entry["label_stealthgpt"] = None

    # 判定 pengram_human_verdict
    try:
        likelihood_human = entry["pengram_human_verdict"]["ai_likelihood"]
        entry["label_human"] = 1 if likelihood_human > THRESHOLD else 0
    except (KeyError, TypeError):
        pass  # 保持为 None

    # 判定 pengram_stealthgpt_verdict
    try:
        likelihood_stealth = entry["pengram_stealthgpt_verdict"]["ai_likelihood"]
        entry["label_stealthgpt"] = 1 if likelihood_stealth > THRESHOLD else 0
    except (KeyError, TypeError):
        pass  # 保持为 None

# 保存结果
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"✅ 已完成标签添加，输出文件：{output_file}")


✅ 已完成标签添加，输出文件：/content/drive/MyDrive/detection/pre2020_human_text_by_genre/output_withStealthgpt_pangram_default.json


In [9]:
# Type1&2 error stats
import json
import pandas as pd
import numpy as np
import scipy.stats as st

# 1. 加载数据
input_path = "/content/drive/MyDrive/detection/pre2020_human_text_by_genre/output_withStealthgpt_pangram_default.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 2. 构建 records，每条样本拆成两条记录：human 与 AI
records = []
for item in data:
    genre = item.get("genre", "unknown")

    # human 版本的预测（ground truth 是 human）
    if "label_human" in item:
        records.append({
            "genre": genre,
            "detector": "pengram",
            "is_human": True,
            "verdict": item["label_human"]
        })

    # stealthgpt 改写版的预测（ground truth 是 AI）
    if "label_stealthgpt" in item:
        records.append({
            "genre": genre,
            "detector": "pengram",
            "is_human": False,
            "verdict": item["label_stealthgpt"]
        })

df = pd.DataFrame(records)

# 3. 置信区间计算函数
def wilson_ci(k, n, alpha=0.05):
    if n == 0:
        return (0, 0)
    z = st.norm.ppf(1 - alpha / 2)
    phat = k / n
    denom = 1 + z**2 / n
    center = phat + z**2 / (2 * n)
    pm = z * ((phat * (1 - phat) / n + z**2 / (4 * n**2)) ** 0.5)
    return ((center - pm) / denom, (center + pm) / denom)

# 4. 评估：错误率 + 精确率 + 召回率
results = []
metrics_results = []

for (genre, detector), group in df.groupby(["genre", "detector"]):
    human = group[group["is_human"]]
    ai = group[~group["is_human"]]

    total_human = len(human)
    total_ai = len(ai)

    fp = (human["verdict"] == 1).sum()
    tn = (human["verdict"] == 0).sum()
    tp = (ai["verdict"] == 1).sum()
    fn = (ai["verdict"] == 0).sum()

    type1_rate = fp / total_human if total_human else np.nan
    type1_ci = wilson_ci(fp, total_human)
    type1_p = st.binomtest(fp, total_human, p=0.5, alternative='greater').pvalue if total_human else np.nan

    type2_rate = fn / total_ai if total_ai else np.nan
    type2_ci = wilson_ci(fn, total_ai)
    type2_p = st.binomtest(fn, total_ai, p=0.5, alternative='greater').pvalue if total_ai else np.nan

    precision = tp / (tp + fp) if (tp + fp) > 0 else np.nan
    precision_ci = wilson_ci(tp, tp + fp) if (tp + fp) > 0 else (np.nan, np.nan)

    recall = tp / (tp + fn) if (tp + fn) > 0 else np.nan
    recall_ci = wilson_ci(tp, tp + fn) if (tp + fn) > 0 else (np.nan, np.nan)

    # 错误率结果
    results.append({
        "Genre": genre,
        "Detector": detector,
        "Error Type": "Type I (False Positive)",
        "Count": fp,
        "Total": total_human,
        "Error Rate": type1_rate,
        "CI Lower": type1_ci[0],
        "CI Upper": type1_ci[1],
        "p-value": type1_p
    })
    results.append({
        "Genre": genre,
        "Detector": detector,
        "Error Type": "Type II (False Negative)",
        "Count": fn,
        "Total": total_ai,
        "Error Rate": type2_rate,
        "CI Lower": type2_ci[0],
        "CI Upper": type2_ci[1],
        "p-value": type2_p
    })

    # Precision & Recall 结果
    metrics_results.append({
        "Genre": genre,
        "Detector": detector,
        "Metric": "Precision",
        "Value": precision,
        "CI Lower": precision_ci[0],
        "CI Upper": precision_ci[1],
        "True Positives": tp,
        "False Positives": fp,
        "Total Detected as AI": tp + fp
    })
    metrics_results.append({
        "Genre": genre,
        "Detector": detector,
        "Metric": "Recall",
        "Value": recall,
        "CI Lower": recall_ci[0],
        "CI Upper": recall_ci[1],
        "True Positives": tp,
        "False Negatives": fn,
        "Total AI": tp + fn
    })


# results_df = pd.DataFrame(results)
metrics_df = pd.DataFrame(metrics_results)




In [10]:
metrics_df

,Genre,Detector,Metric,Value,CI Lower,CI Upper,True Positives,False Positives,Total Detected as AI,False Negatives,Total AI
0,amazon review,pengram,Precision,0.994681,0.980816,0.998540,374,2.0,376.0,NaN,NaN
1,amazon review,pengram,Recall,0.939698,0.911844,0.959146,374,NaN,NaN,24.0,398.0
2,blog,pengram,Precision,1.000000,0.992126,1.000000,484,0.0,484.0,NaN,NaN
3,blog,pengram,Recall,0.973843,0.955766,0.984651,484,NaN,NaN,13.0,497.0
4,news,pengram,Precision,0.966197,0.950196,0.977181,686,24.0,710.0,NaN,NaN
5,news,pengram,Recall,0.994203,0.985190,0.997743,686,NaN,NaN,4.0,690.0
6,restaurant review,pengram,Precision,1.000000,0.989442,1.000000,360,0.0,360.0,NaN,NaN
7,restaurant review,pengram,Recall,0.932642,0.903136,0.953622,360,NaN,NaN,26.0,386.0
8,resume,pengram,Precision,1.000000,0.991000,1.000000,423,0.0,423.0,NaN,NaN
9,resume,pengram,Recall,1.000000,0.991000,1.000000,423,NaN,NaN,0.0,423.0


In [11]:
metrics_df.to_csv("/content/drive/MyDrive/detection/pre2020_human_text_by_genre/Stealthgpt_pangram_default.csv")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
